In [1]:
import pandas as pd 
import numpy as np

In [2]:
df=pd.read_csv("TRAIN.CSV")
df.head()

,AT,V,AP,RH,PE
0,22.85,60.29,1017.29,61.92,448.58
1,11.75,35.76,1018.99,57.92,473.01
2,12.24,44.90,1020.31,82.22,464.25
3,13.43,40.10,1015.82,87.90,471.77
4,4.65,35.19,1018.23,94.78,489.36


In [3]:
df.shape

(8000, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      8000 non-null   float64
 1   V       8000 non-null   float64
 2   AP      8000 non-null   float64
 3   RH      8000 non-null   float64
 4   PE      8000 non-null   float64
dtypes: float64(5)
memory usage: 312.6 KB


In [5]:
Y=df["PE"]
X=df.drop(["PE","RH"],axis=1)

X.shape,Y.shape

((8000, 3), (8000,))

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
from xgboost import XGBRegressor
xgb=make_pipeline(StandardScaler(), XGBRegressor(colsample_bytree=0.7, 
                                                 learning_rate=0.05, max_depth=None,
                                                 min_child_weight=4, n_estimators=500, nthread=4, 
                                                 objective='reg:linear',subsample=0.7))
xgb.fit(X,Y)

[10:41:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.7, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.05,
                              max_delta_step=0, max_depth=6, min_child_weight=4,
                              missing=nan, monotone_constraints='()',
                              n_estimators=500, n_jobs=4, nthread=4,
                              num_parallel_tree=1, objective='reg:linear',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.7,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [8]:
train_df=pd.read_csv("TEST.csv")
train_df.head()

,AT,V,AP,RH
0,11.55,40.60,1014.44,87.06
1,6.61,38.91,1015.77,92.31
2,16.97,42.86,1013.92,74.80
3,21.90,57.76,1017.45,66.89
4,9.73,40.22,1011.48,87.88


In [9]:
X_test=train_df.drop(["RH"],axis=1)
X_test.head()

,AT,V,AP
0,11.55,40.60,1014.44
1,6.61,38.91,1015.77
2,16.97,42.86,1013.92
3,21.90,57.76,1017.45
4,9.73,40.22,1011.48


In [10]:
xgb_pred=xgb.predict(X_test)

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [11]:
submission = pd.DataFrame({
        "Value":xgb_pred
        })
submission.head()
#To save file - 
submission.to_csv('xgb_submission.csv', index=True) 

In [12]:
'''
from sklearn.model_selection import GridSearchCV

xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,Y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)'''

"\nfrom sklearn.model_selection import GridSearchCV\n\nxgb1 = XGBRegressor()\nparameters = {'nthread':[4], #when use hyperthread, xgboost may become slower\n              'objective':['reg:linear'],\n              'learning_rate': [.03, 0.05, .07], #so called `eta` value\n              'max_depth': [5, 6, 7],\n              'min_child_weight': [4],\n              'silent': [1],\n              'subsample': [0.7],\n              'colsample_bytree': [0.7],\n              'n_estimators': [500]}\n\nxgb_grid = GridSearchCV(xgb1,\n                        parameters,\n                        cv = 2,\n                        n_jobs = 5,\n                        verbose=True)\n\nxgb_grid.fit(X,Y)\n\nprint(xgb_grid.best_score_)\nprint(xgb_grid.best_params_)"